In [1]:
import json
import numpy as np
from tqdm import tqdm
import re
import os
import json
import random

In [2]:
q_regex = re.compile(r"^\W{0,5}\d{1,2}\. |^\W{0,5}Question\s?\d{0,2}\W{0,5}|^\W{0,5}Q\s?\d{1,2}\W{0,5}|^\W{0,5}\d{1,2}\. Question\W{0,5}")
a_regex = re.compile(r"^\W{0,5}\d{0,2}\W{0,5}Answer\s?\d{0,2}\W{0,5}|^-|^\d{0,2}\W{0,5}A\s?\d{0,2}\W{0,5}:")


def tuplize_qa_pair(q, a):
    a=a.strip()
    if len(a)>10:
        if a_regex.match(a):
            a_preface, a_content = a_regex.split(a,1)
        elif a[0].isalpha():
            a_content = a
        else:
            raise RuntimeError(f"Answer regex incomplete: {q}\nANSWER:{a}")
        
        a_content = a_content.strip()
        if len(a_content)<10:
            print(f"Small answer dropping: {a_content}")
            return None
        return (q, a_content)
    else:
        if len(a)<10 and len(a)>0:
            print(f"Small answer dropping: {a}")
        return None


def one_example(sample):
    sample_dict = json.loads(sample)
    qa_lines = sample_dict["messages"][2]["content"].split("\n")
    qa_pair_list = []
    q_prefix = ''
    q_content = ''
    a = ''
    missing = 0
    for line in qa_lines:
        line = line.strip()
        if q_regex.match(line) and not a_regex.match(line):
            old_q_prefix = q_prefix
            old_q_content = q_content
            assert len(q_regex.findall(line))==1, q_regex.findall(line)
            q_prefix = q_regex.findall(line)[0]
            q_content  = q_regex.split(line,1)[1]
            if q_prefix == old_q_prefix and re.match(r"\d",q_prefix):
                # Answer recovery when q and a are both 1. 2. etc
                a+=q_content+"\n"
                q_content = old_q_content
            else:
                qa_pair = tuplize_qa_pair(q_content, a)
                if qa_pair is not None:
                    qa_pair_list.append(qa_pair)
                a = ''
        else:
            a+=line+'\n'
    qa_pair = tuplize_qa_pair(q_content, a)
    if qa_pair is not None:
        qa_pair_list.append(qa_pair)
    return qa_pair_list, 10-len(qa_pair_list)

In [3]:
image_dir = "/hdd/LLM/limuBERT_data/train_llasa/images"

In [4]:
def limubert_normalize(data):
    data[:, :, :3] = data[:, :, :3]/9.8
    return data

In [5]:
training_jsonl_filepath = "llasa_training_v1.jsonl"

root_data_dir = "/hdd/LLM/limuBERT_data/extracted_data"

data_file_name = "data_20_120.npy"
label_file_name = "label_20_120.npy"


with open('/hdd/LLM/limuBERT_data/dataset_activity_label.json') as json_file:
    dataset_activity_label_dict = json.load(json_file)


last_axis_dict = {
    "hhar": 2, "motion":0, "shoaib":0, "uci":0
}
datasets = last_axis_dict.keys()

all_qa_pair = []
qa_pair_list = []
for dataset in datasets:
    p = 0
    sample_count = 0
    with open(f"/hdd/LLM/opensqa_10hz_{dataset}.jsonl","r") as f:
        samples = f.readlines()
    sensor_data = np.load(os.path.join(root_data_dir, dataset, data_file_name))
    sensor_data = limubert_normalize(sensor_data)
    
    for sample in tqdm(samples):
        qa_pairs, missing = one_example(sample)
        assert missing>=0, f"Extra: {qa_pairs} Missing: {missing}"
        if missing>5:
            print(f"{missing} missing:{json.loads(sample)['messages'][2]['content']}")
            print(qa_pairs)
            if missing<10:
                print("Potential corner case")
        else:
            # accl, gyro = sensor_data[sample_count][:, 0:3], sensor_data[sample_count][:, 3:6] # acel, gyro seperate
            accl_gyro = sensor_data[sample_count][:,:6] # accl, gyro together
            all_qa_pair.extend(qa_pairs)
            for i, (q, a) in enumerate(qa_pairs):
                sensor_sample_id = f"{sample_count}_{i}"
                sensor_sample_filepath = os.path.join(image_dir , f'{dataset}/sensor_image_{sample_count}.npy')
                np.save(sensor_sample_filepath, accl_gyro)
                qa_pair = {
                    "id": sensor_sample_id,
                    "image": sensor_sample_filepath, # Sensor csv
                    "conversations": [
                        {"from": "human", "value": f"<image>\n{q}"}, # question
                        {"from": "gpt", "value": a}, # answers
                    ],
                }
                qa_pair_list.append(qa_pair)
                
        sample_count += 1
        p+=missing
    print(f"Missing: {p}, {100*p/(len(sensor_data)*10)}%")

  7%|▋         | 666/9166 [00:01<00:13, 614.37it/s]

10 missing:1. Based on the gyroscope data showing changes in all axes and the indication of turning and maneuvering, can you deduce if the user is riding on a straight road or a curved path? 
2. Given the gyroscope z-axis vibrations coincide with road bumps, how can you differentiate between different types of road conditions, such as riding on a smooth road versus a bumpy off-road trail, using this information?
3. How can the accelerometer readings help in determining the intensity of pedaling during biking based on the repetitive motion patterns observed?
4. Can the sudden spikes in accelerometer data be used to determine if the user is braking, accelerating, or encountering obstacles while biking? 
5. How do the gyroscope x-axis values being high and steady correlate with the user's speed maintenance during biking? 
6. Considering the gyroscope data and the accelerometer z-axis fluctuations, how could you infer whether the user is cycling on flat terrain or encountering varying elev

 15%|█▍        | 1365/9166 [00:02<00:12, 619.81it/s]

10 missing:1. Based on the gyroscope data, can we determine the direction in which the biker is leaning during the biking activity? If so, how can we infer this information from the sensor readings?
2. How can we use the accelerometer data to differentiate between uphill and downhill segments of the biking route?
3. Is it possible to estimate the speed of the biker during specific instances of the biking session using the gyroscope and accelerometer data? If yes, how can we calculate this speed?
4. Can we identify any specific maneuvers performed by the biker, such as sharp turns or sudden stops, by analyzing both the gyroscope and accelerometer data together?
5. How do the gyroscope and accelerometer readings change when the biker accelerates from a stationary position at the beginning of the biking activity?
6. Are there any patterns or trends in the gyroscope data that indicate when the biker is pedaling harder or softer during the biking session?
7. Can we determine the type of sur

 19%|█▉        | 1727/9166 [00:02<00:12, 582.41it/s]

...

Small answer dropping: 1.


 83%|████████▎ | 1739/2088 [00:03<00:00, 541.16it/s]

Small answer dropping: Yes
10 missing:1. How can the abrupt movements observed during the descent of stairs be identified in the gyroscope data?
2. How do the fluctuations in the accelerometer readings align with the user's preparation to start descending stairs?
3. Can the varying tilts and rotations in the gyroscope data be correlated with specific steps taken during the descent?
4. How do the sudden spikes in accelerometer values correspond to the foot landing on each step during the descent of stairs?
5. In what ways do the gyroscope readings reflect the user's adjustment to different steps while descending stairs?
6. How does the consistent downward trend in the z-axis of the accelerometer data indicate the user's continuous descent down the stairs?
7. What role do the rapid changes in orientation in the gyroscope data play in the user's navigation through each step during the descent?
8. How do the occasional pauses observed in the accelerometer data affect the overall pattern of

100%|██████████| 2088/2088 [00:03<00:00, 526.77it/s]

Missing: 348, 1.6666666666666667%


In [6]:
with open(training_jsonl_filepath, "w") as outfile:
    json.dump(qa_pair_list, outfile)

In [7]:
all_qa_pair[50000]

('How can we determine the energy expenditure or effort exerted by the user while climbing stairs based on the provided sensor data?',
 'Moments of rest or pause during the stair climbing activity can be identified from the gyroscope and accelerometer data by looking for periods of minimal or stable movement. In the gyroscope data, these moments would appear as consistent values with minimal fluctuations, indicating a pause in orientation adjustments. Simultaneously, in the accelerometer data, moments of rest or pause would be characterized by steady acceleration values in all axes, signifying little to no vertical motion. By detecting these stable intervals in both sensor datasets, we can pinpoint moments of rest or pause during the stair climbing activity.')